# Model evaluation
This is the final script, which evaluates the final model using the validation set, which has not been used up until now.

## 0. Setup

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
export_folder = f'export'
date_format = "%d%m%Y%H%M%S" # timestamp format in exported files

In [ ]:
model_path = "git-ignored-models/model_17112022221102.h5" 

In [ ]:
base_path = "data" 
img_folder_valid = base_path + "/valid"
groundtruth_file_valid = base_path + "/ISIC_2020_2019_valid.csv"
num_cores = 8

## 1. Load the model

In [ ]:
from tensorflow.keras.models import load_model
from efficientnet.tfkeras import EfficientNetB0

model = load_model(model_path)

In [ ]:
from math import sqrt

input_shape = model.get_config()["layers"][0]["config"]["batch_input_shape"]
expect_flattened_input = len(input_shape) == 2
img_pixel = int(sqrt(input_shape[1]/3) if expect_flattened_input else input_shape[1])

are_flat = "are" if expect_flattened_input else "are not"
print(f"the model expects images with size {img_pixel}*{img_pixel} which {are_flat} flattened out")

## 2. Load the data

In [ ]:
from utilities import get_all_img_paths

img_paths_valid = get_all_img_paths(img_folder_valid) 

In [ ]:
from utilities import get_df

result = [None]
num_images_per_thread = int(len(img_paths_valid) / num_cores)
print("Num images per thread %d"%num_images_per_thread)

get_df(result, 0, img_paths_valid, groundtruth_file_valid, "valid", num_images_per_thread, img_pixel)

df_valid = result[0]

In [ ]:
from utilities import split_predictors_target

X_valid, y_valid = split_predictors_target(df_valid) 

In [ ]:
if not expect_flattened_input:
    from utilities import unflatten_images_df
    X_valid = unflatten_images_df(X_valid, img_pixel=img_pixel)

## 3. Predict the validation set

In [ ]:
y_pred_continuous = model.predict(X_valid)
y_pred_discrete = (model.predict(X_valid) > 0.5).astype("int32")
y_pred = y_pred_discrete

## 4. Evaluate the results

In [ ]:
_, valid_acc, valid_recall = model.evaluate(X_valid, y_valid)
print('Accuracy\tValid: %.3f' % (valid_acc))
print('Recall\tValid: %.3f' % (valid_recall))

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_val, y_pred, digits=4)
print(f'\nClassification_report=\n{report}')

file = open(export_folder + "/classification_report_"+datetime.datetime.now().strftime(date_format)+".txt", 'w')
file.write(report)
file.close()

In [ ]:
from matplotlib import pyplot
import seaborn as sns
from sklearn.metrics import confusion_matrix

class_names = ["no melanoma", "melanoma"]

cf = confusion_matrix(y_valid, y_pred)
plot = sns.heatmap(cf, annot= True, fmt=".0f",
           xticklabels = class_names,
           yticklabels = class_names)
plot.set(xlabel='Prediction', ylabel='Actual')

plot.get_figure().savefig(export_folder + '/confusion_matrix_' + datetime.datetime.now().strftime(date_format) + ".png")

In [ ]:
from utilities import display_interesting_results

plt_wrong = display_interesting_results(X_valid, y_pred, y_valid, img_pixel=img_pixel, flat=expect_flattened_input)
plt_wrong.savefig(export_folder + "/incorrect_classification_results_"+datetime.datetime.now().strftime(date_format)+".png")
plt_wrong.show()